In [1]:
import os
import sys
import json
sys.path.append(os.path.join(os.getcwd().replace("model_inference", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.profiler import profiler
import matplotlib.pyplot as plt
import time
from parse_dataset import NetworkDataset, parse_dataset, split_datasets, binary_dataset
from split_model import SplitModelDPU, SplitModelHost
from load_models import models
from transfer_tensors import DPUSocket

In [2]:
conf = {
    "batch_size": 128,
    "epochs": 10,
    "learning_rate": 0.0001,
    "dpu": True
}

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
dataset_path = os.path.join(os.getcwd().replace("model_inference", ""), "datasets")

# load label_dict
json_file = os.path.join(dataset_path, "label_index.json")
with open(json_file, 'r') as file:
    label_dict = json.load(file)

# load train, val and test datasets
train_dataset_file = os.path.join(dataset_path, "train_dataset.pt")
X_train, y_train = torch.load(train_dataset_file)

val_dataset_file = os.path.join(dataset_path, "val_dataset.pt")
X_val, y_val = torch.load(val_dataset_file)

test_dataset_file = os.path.join(dataset_path, "test_dataset.pt")
X_test, y_test = torch.load(test_dataset_file)

X_train, X_val, X_test = X_train.unsqueeze(-1), X_val.unsqueeze(-1), X_test.unsqueeze(-1)
print(X_train.shape)
print(y_train.shape)

# create train, val and test datasets
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=conf["batch_size"], shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=conf["batch_size"], shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=conf["batch_size"])

torch.Size([356334, 513, 1])
torch.Size([356334])


In [5]:
class BenchmarkDPU:
    def __init__(self, dpu_model, dpu_model_path):
        self.dpu_model = dpu_model
        self.dpu_model.load(dpu_model_path)
    
    def run(self, loader):
        """Run benchmark measuring model accuracy, cpu time, memory usage and total runtime"""
        self.dpu_model.model.eval()

        # warm-up before profiling:
        for i in range(5):
            with torch.no_grad():
                data, labels = next(iter(loader))
                pred, _ = self.dpu_model.model(data)

        with torch.profiler.profile(
            activities=[profiler.ProfilerActivity.CPU],
            record_shapes=True,
            profile_memory=True,
            with_stack=True
        ) as prof:
            data, labels = next(iter(loader))
            bin_labels = binary_dataset(labels, label_dict)
            start = time.perf_counter()
            # warm-up before profiling:
            with torch.no_grad():
                pred, logits = self.dpu_model.model(data)
                logits = logits.detach()  # break the graph here
                logits = logits.unsqueeze(1)
            end = time.perf_counter()
        
        accuracy = (pred.round() == bin_labels).float().mean()
        
        # Wall-clock time
        wall_clock_time = end - start

        # CPU compute time (µs → s)
        cpu_time_total_s = sum([e.self_cpu_time_total for e in prof.key_averages()]) / 1e6  # microseconds
        cpu_utilization = cpu_time_total_s / wall_clock_time

        # peak memory during profiling
        peak_mem = max([e.cpu_memory_usage for e in prof.key_averages()])
        peak_mem = peak_mem / 1024**2 #MB

        return 100*accuracy, cpu_utilization, peak_mem, wall_clock_time, logits, labels

In [6]:
dpu_path = os.path.join(os.getcwd().replace("model_inference", ""), "checkpoint", "dpu_split_model.pth")
dpu_model = models["dpu"]
# dpu_model.load(dpu_path)

In [7]:
benchmark = BenchmarkDPU(dpu_model, dpu_path)

Checkpoint loaded from /home/ubuntu/Network-Packet-ML-Model/checkpoint/dpu_split_model.pth!


In [8]:
so_file = os.path.join(os.getcwd().replace("model_inference", ""), "socket_transfer", "socket_transfer.so")
address = "10.128.14.17"
socket = DPUSocket(so_file, address)

In [9]:
acc, cpu, mem, runtime, logits, targets = benchmark.run(test_loader)

[W1024 10:02:48.149626067 CPUAllocator.cpp:245] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event


In [10]:
print(f"Benchmark (dpu): Accuracy: {acc:.2f}%, CPU Usage: {cpu:.2f} cores, Memory Usage: {mem:.2f}MB, Runtime: {runtime:.2f}s")

Benchmark (dpu): Accuracy: 66.71%, CPU Usage: 1.00 cores, Memory Usage: 384.81MB, Runtime: 4.32s


In [11]:
# send logits
socket.send(logits)

Buffer of size 65536 bytes sent to host! 


In [12]:
#send targets
targets = targets.to(dtype=torch.float)
socket.send(targets)

Buffer of size 512 bytes sent to host! 


In [13]:
print(logits[0])
print(targets[:10])
print(logits.shape, targets.shape)

tensor([[ 0.7089, -0.7476, -0.0036,  1.4039, -0.9515,  0.7841,  1.7679, -0.7348,
          0.9583, -0.7040, -1.2197, -0.9904, -0.7988,  0.2641, -0.5364, -0.3622,
          0.7251, -0.8590,  0.7065,  0.7263, -0.7797, -0.3498, -0.6212,  0.8428,
          0.8782,  0.7267, -0.9448, -0.7076,  0.8949, -0.8892,  0.7929,  0.2626,
         -0.6879,  0.7814,  0.7038,  0.4580,  1.3909,  0.7223, -0.7195,  0.6840,
         -0.7028, -0.7435, -0.8354,  0.8158,  0.7670, -1.2488, -0.8105,  0.8674,
         -0.5246,  0.7223, -1.3902,  1.2885,  0.6975, -0.4087, -0.6764, -1.6874,
          0.8298, -0.6905,  0.7101, -0.7583, -0.8911, -0.6338,  0.9207,  0.8254,
         -0.0968,  0.8207,  0.7985,  0.7873, -0.3857, -0.3739, -0.3412,  1.2092,
         -0.5098, -0.3415,  1.1584, -0.3423,  1.3440,  0.5424,  1.2486,  0.7654,
          0.4450, -0.4075,  0.0813,  0.8742,  1.1671,  0.4403, -0.2603,  0.5596,
         -1.2576,  0.6712,  0.3430,  0.5415, -0.0528, -1.0890,  0.3648, -1.1970,
         -1.2779,  0.3953,  